# Computing the Cohen's d measure for Jacobians

In [ ]:
#!pip install pingouin==0.3.10 --user
import numpy as np
from numpy import std, mean, sqrt
import pingouin as pg
import pandas as pd
import nibabel as nib
import os

Computing the Cohen's d manually and testing the function

In [ ]:
def cohend_func(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

In [ ]:
x = [1, 2, 3, 4, 5]
y = [3, 4, 5, 6, 7, 8]
print(pg.compute_effsize(y, x, eftype='cohen'))
print(cohend_func(y,x))

Worked!
<br>Copying functions from RAG2_cohend.ipynb

In [12]:
def nifti_to_vector(nii_file):
    '''
    Takes an input file address (str) to the NIFTI file and returns
    the flattened 1D array and the shape of the input matrix.
    
    Parameters:
    nii_file (str): The address to nii file
    
    Returns:
    vector (np.array): The flattened 1D array
    shape (list): Shape of the original matrix
    '''
    img = nib.load(nii_file)
    array = np.array(img.dataobj)
    shape = array.shape
    vector = array.flatten()
    return vector, shape

In [21]:
vector = nifti_to_vector('/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/MKO_invjcb1.nii')

In [25]:
vector[0]

array([0.0000000e+00, 2.3841858e-07, 2.3841858e-07, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [26]:
def stack_nii(path_prefix, n):
    '''
    Takes the path to NIFTI files and stacks them in rows.
    
    Parameters:
    path_prefix (str): Prefix of the path to the NIFTI file until the number,
        example: '/data/img' if the complete location is: '/data/img1.nii'
    
    n (int): Number of NIFTI files to input
    
    Returns:
    stack (array): Numpy array with n rows
    '''
    input_list = [path_prefix+str(i)+'.nii' for i in range(1,n+1)]
    stack = np.vstack([nifti_to_vector(input)[0]] for input in input_list)
    return stack

In [27]:
MKO_stack = stack_nii('/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/MKO_invjcb',5)

/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


In [28]:
MWT_stack = stack_nii('/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/MWT_invjcb',5)

/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


In [30]:
def cohen_d(stack1, stack2):
    '''
    Takes 2 arrays and computes then Cohen's d using the cohend_func function
    
    Parameters:
    stack1, stack2 (array): Arrays with subjects as rows and voxels as columns
    
    Returns:
    d (vector): Vector with size of stack1/stack2 columns
    '''
    d = np.zeros(stack1.shape[1])
    for i in range (stack1.shape[1]):
        d[i] = cohend_func(stack1[:,i], stack2[:,i])
    return d

In [31]:
d = cohen_d(MKO_stack, MWT_stack)

/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [34]:
shape

(168, 265, 155)

In [35]:
d_rs = d.reshape((168,265,155))

In [39]:
def array_to_nifti(arr,output_file):
    """
    Takes an array and saves it as NIFTI file in the given output address
    
    Parameters:
    arr (array): matrix of image voxels
    output_file (string): output file address
    
    Returns:
    None
    """
    
    save = nib.Nifti1Image(arr, np.eye(4))
    save.header.get_xyzt_units()
    save.to_filename(os.path.join('build',output_file))

In [40]:
array_to_nifti(d_rs, '/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/RAG2_cohensd.nii')

In [41]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/
3dinfo -orient -ad3 -space MKO_invjcb1.nii
3dinfo -orient -ad3 -space RAG2_cohensd.nii

LPI	0.075000	0.075000	0.075000	ORIG
LPI	1.000000	1.000000	1.000000	TLRC


3dinfo: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
3dinfo: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/RAG2_cohensd.nii to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.


In [42]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/
3drefit -xyzscale 0.075 RAG2_cohensd.nii
3drefit -space ORIG RAG2_cohensd.nii
3drefit -view orig RAG2_cohensd.nii

3drefit: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
++ 3drefit: AFNI version=AFNI_20.1.11 (May 28 2020) [64-bit]
++ Authored by: RW Cox
** AFNI converts NIFTI_datatype=64 (FLOAT64) in file /data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/RAG2_cohensd.nii to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Processing AFNI dataset RAG2_cohensd.nii
 + applying -xyzscale
 + deoblique
 + loading and re-writing dataset RAG2_cohensd.nii (/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/RAG2_cohensd.nii in NIFTI storage)
++ 3drefit processed 1 datasets
3drefit: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
++ 3drefit: AFNI version=AFNI_20.1.11 (May 28 2020) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset RAG2_cohensd.nii
*+ WARNING: Changing the space of a TLRC view data

In [43]:
%%bash
module load afni/20.1.11
cd /data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/
@Align_Centers -cm -overwrite -base C57B_avg75.nii -dset RAG2_cohensd.nii

base: -6.38487 -8.60393 4.10522
dset: -83.7791 -130.269 76.6169
delta: 77.394230 121.665070 -72.511680


ParseName: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
ParseName: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
afni: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
afni: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
afni: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
afni: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
ParseName: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
3dCM: /packages/7x/anaconda3/5.3.0/lib/libuuid.so.1: no version information available (required by /lib64/libSM.so.6)
3dcopy: /packages/7x/anaconda3/5.3.0/lib/

In [53]:
def copy_nifti_header(base, input, outfile):
    """
    Takes the NIFTI header from base file and overwrites it for the input
    (Nibabel library needed)
    
    Parameters:
    base (str): base file address, its header gets copied to input's
    input (str): input file address, its header gets replaced with base's
    outfile (str): output file address
    
    Returns:
    None
    """
    
    base_img = nib.load(base)
    
    input_img = nib.load(input)
    input_matrix = np.array(input_img.dataobj).astype('float32')
    
    copy_img = nib.Nifti1Image(input_matrix, header=base_img.header, affine=base_img.affine)
    nib.save(copy_img, outfile)

In [46]:
array_to_nifti(d_rs, '/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/test_copy.nii')

In [52]:
base = '/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/RAG2_cohensd_shft.nii'
input = '/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/test_copy.nii'
outfile = '/data/bioprotean/RAG2/AVG/C57B/Jacobian/JCB/copied.nii'
copy_nifti_header(base, input, outfile)